In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# Binance

In [ ]:
from ccstabilizer import Exchange
from ccstabilizer import secrets

In [ ]:
# export
from decimal import Decimal


class Binance(Exchange):

    def __init__(self):
        super().__init__()
        self.binance = BinanceAPI(os.environ['BINANCE_API_KEY'], os.environ['BINANCE_API_SECRET'])
        for symbol_info in self.binance.exchangeInfo().get('symbols', []):
            symbol = symbol_info.get('symbol', '')
            trading_spec = {}
            for binance_filter in symbol_info.get('filters', []):
                if binance_filter.get('filterType', '') == 'LOT_SIZE':
                    trading_spec['min_trade_unit'] = Decimal(binance_filter.get('stepSize', '0.0001'))
                    break
            for binance_filter in symbol_info.get('filters', []):
                if binance_filter.get('filterType', '') == 'MIN_NOTIONAL':
                    trading_spec['min_trade_fiat_money_limit'] = Decimal(binance_filter.get('minNotional', '0'))
                    break
            trading_spec['fee_rate'] = Decimal('0.001')
            if 'permissions' in symbol_info:
                trading_spec['liquid'] = 'SPOT' in symbol_info.get('permissions', [])
            self.trading_specifications[symbol] = trading_spec

    def get_trading_specification(self, crypto_symbol, fiat_symbol):
        return self.trading_specifications.get(f'{crypto_symbol}{fiat_symbol}', {})

    def get_portfolio(self):
        return {
            asset.get('asset', ''): Decimal(asset.get('free', '0')) for asset in self.binance.account().get('balances', [])
        }

    def get_price(self, crypto_symbol, fiat_symbol):
        avg_price = self.binance.avgPrice(symbol=f'{crypto_symbol}{fiat_symbol}')
        if 'price' in avg_price:
            self.get_trading_specification(crypto_symbol, fiat_symbol)['average_fiat_price'] = Decimal(avg_price.get('price', '0'))
        now_ticker = self.binance.tickerBookTicker(symbol=f'{crypto_symbol}{fiat_symbol}')
        now_buy_fiat_price_without_fee = Decimal(now_ticker.get('askPrice', 'Infinity'))
        now_sell_fiat_price_without_fee = Decimal(now_ticker.get('bidPrice', '0'))
        fee_rate = self.get_trading_specification(crypto_symbol, fiat_symbol).get('fee_rate', 0)
        return {
            'now_buy_fiat_price': self.get_buy_fiat_price(now_buy_fiat_price_without_fee, fee_rate),
            'now_sell_fiat_price': self.get_sell_fiat_price(now_sell_fiat_price_without_fee, fee_rate),
            'now_buy_fiat_price_without_fee': now_buy_fiat_price_without_fee,
            'now_sell_fiat_price_without_fee': now_sell_fiat_price_without_fee,
        }

    def get_buy_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee / (1 - fee_rate)

    def get_sell_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee * (1 - fee_rate)

    def buy(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee=None):
        if amount <= 0:
            raise Exception('Binance::buy  amount <= 0')
        self.binance.createOrder(
            symbol=f'{crypto_symbol}{fiat_symbol}', side='BUY', type='MARKET', quantity=str(amount), recvWindow=5000
        )

    def sell(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee=None):
        if amount <= 0:
            raise Exception('Binance::sell  amount <= 0')
        self.binance.createOrder(
            symbol=f'{crypto_symbol}{fiat_symbol}', side='SELL', type='MARKET', quantity=str(amount), recvWindow=5000
        )

    def has_enough_unused_fiat_money(self, buy_fiat_money, unused_fiat_money):
        return buy_fiat_money <= unused_fiat_money

    def is_trade_fiat_money_larger_than_limit(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee=None):
        # Just for Binance
        average_fiat_price = self.get_trading_specification(crypto_symbol, fiat_symbol).get('average_fiat_price', 0)
        return average_fiat_price * amount >= self.get_trading_specification(crypto_symbol, fiat_symbol).get('min_trade_fiat_money_limit', 0)

## Binance API
https://github.com/binance/binance-spot-api-docs/blob/master/rest-api_CN.md

In [ ]:
# export
# Credits to @Bablofil https://github.com/Bablofil/binance-api
import contextlib
import time
import json
import urllib
import hmac, hashlib
import requests

from urllib.parse import urlparse, urlencode
from urllib.request import Request, urlopen

class BinanceAPI(object):

    methods = {
            #  Public methods
            'ping':             {'url': 'ping', 'method': 'GET', 'private': False},
            'time':             {'url': 'time', 'method': 'GET', 'private': False},
            'exchangeInfo':     {'url': 'exchangeInfo', 'method': 'GET', 'private': False},
            'depth':            {'url': 'depth', 'method': 'GET', 'private': False},
            'trades':           {'url': 'trades', 'method': 'GET', 'private': False},
            'historicalTrades': {'url': 'historicalTrades', 'method': 'GET', 'private': False},
            'aggTrades':        {'url': 'aggTrades', 'method': 'GET', 'private': False},
            'klines':           {'url': 'klines', 'method': 'GET', 'private': False},
            'avgPrice':         {'url': 'avgPrice', 'method': 'GET', 'private': False},
            'ticker24hr':       {'url': 'ticker/24hr', 'method': 'GET', 'private': False},
            'tickerPrice':      {'url': 'ticker/price', 'method': 'GET', 'private': False},
            'tickerBookTicker': {'url': 'ticker/bookTicker', 'method': 'GET', 'private': False},
            #  Private methods
            'createOrder':      {'url': 'order', 'method': 'POST', 'private': True},
            'testOrder':        {'url': 'test', 'method': 'POST', 'private': True},
            'orderInfo':        {'url': 'order', 'method': 'GET', 'private': True},
            'cancelOrder':      {'url': 'order', 'method': 'DELETE', 'private': True},
            'openOrders':       {'url': 'openOrders', 'method': 'GET', 'private': True},
            'allOrders':        {'url': 'allOrders', 'method': 'GET', 'private': True},
            'account':          {'url': 'account', 'method': 'GET', 'private': True},
            'myTrades':         {'url': 'myTrades', 'method': 'GET', 'private': True},
    }

    RETRY_INTERVAL = 60

    def __init__(self, API_KEY, API_SECRET):
        self.API_KEY = API_KEY
        self.API_SECRET = bytearray(API_SECRET, encoding='utf-8')
        self.shift_seconds = 0

    def __getattr__(self, name):
        def wrapper(*args, **kwargs):
            kwargs.update(command=name)

            while True:
                try:
                    return self.call_api(**kwargs)
                except Exception as e:
                    print(e)
                    raise
                    time.sleep(type(self).RETRY_INTERVAL)

        return wrapper

    def set_shift_seconds(self, seconds):
        self.shift_seconds = seconds

    def call_api(self, **kwargs):

        command = kwargs.pop('command')
        api_url = 'https://api.binance.com/api/v3/' + self.methods[command]['url']

        payload = kwargs
        headers = {}

        payload_str = urllib.parse.urlencode(payload)
        if self.methods[command]['private']:
            payload.update({'timestamp': int(time.time() + self.shift_seconds - 1) * 1000})
            payload_str = urllib.parse.urlencode(payload).encode('utf-8')
            sign = hmac.new(
                key=self.API_SECRET,
                msg=payload_str,
                digestmod=hashlib.sha256
            ).hexdigest()

            payload_str = payload_str.decode("utf-8") + "&signature="+str(sign) 
            headers = {"X-MBX-APIKEY": self.API_KEY}

        if self.methods[command]['method'] == 'GET':
            api_url += '?' + payload_str

        with contextlib.closing(
            requests.request(
                method=self.methods[command]['method'], url=api_url, headers=headers,
                data='' if self.methods[command]['method'] == 'GET' else payload_str
            )
        ) as response:
            response_json = response.json()
            if 'code' in response_json:
#                 {"code":-1021,"msg":"Timestamp for this request is outside of the recvWindow."}
                raise Exception(response_json.get('msg', ''))
            return response_json

In [ ]:
import pprint
pprint.PrettyPrinter(indent=4).pprint([symbol for symbol in BinanceAPI(os.environ['BINANCE_API_KEY'], os.environ['BINANCE_API_SECRET']).exchangeInfo()['symbols'] if symbol['symbol'] == 'CELOUSDT'])